In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# path="/content/gdrive/My Drive/E4Colaboratory/cnn"


In [0]:
# TRAINING___________________________________________________________________________________________________________________________________________________________

In [0]:
#set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [0]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
from tqdm import tqdm

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K



def buildModel(width, height, depth, classes):
  # initialize the model along with the input shape to be
  # "channels last" and the channels dimension itself
  model = Sequential()
  inputShape = (height, width, depth)
  chanDim = -1

  # if we are using "channels first", update the input shape
  # and channels dimension
  if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    print("yes")
    chanDim = 1

  print("[INFO] 1...")
  # CONV => RELU => POOL
  model.add(Conv2D(32, (3, 3), padding="same",
  input_shape=inputShape))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(3, 3)))
  model.add(Dropout(0.25))

  print("[INFO] 2...")
  # (CONV => RELU) * 2 => POOL
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  print("[INFO] 3...")
  # (CONV => RELU) * 2 => POOL
  model.add(Conv2D(128, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(128, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  print("[INFO] 4...")
  # first (and only) set of FC => RELU layers
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation("relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  print("[INFO] 5...")
  # softmax classifier
  model.add(Dense(classes))
  model.add(Activation("softmax"))

  # return the constructed network architecture
  return model





In [0]:
# def buildModel(width, height, depth, classes):
#   # initialize the model along with the input shape to be
#   # "channels last" and the channels dimension itself
#   model = Sequential()
#   inputShape = (height, width, depth)
#   chanDim = -1

#   # if we are using "channels first", update the input shape
#   # and channels dimension
#   if K.image_data_format() == "channels_first":
#     inputShape = (depth, height, width)
#     print("yes")
#     chanDim = 1

#   print("[INFO] 1...")
#   # CONV => RELU => POOL
#   model.add(Conv2D(32, (3, 3), padding="same",
#   input_shape=inputShape))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization(axis=chanDim))
#   model.add(MaxPooling2D(pool_size=(3, 3)))
#   model.add(Dropout(0.25))

#   print("[INFO] 2...")
#   # (CONV => RELU) * 2 => POOL
#   model.add(Conv2D(64, (3, 3), padding="same"))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization(axis=chanDim))
#   model.add(Conv2D(64, (3, 3), padding="same"))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization(axis=chanDim))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))

#   print("[INFO] 3...")
#   # (CONV => RELU) * 2 => POOL
#   model.add(Conv2D(128, (3, 3), padding="same"))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization(axis=chanDim))
#   model.add(Conv2D(128, (3, 3), padding="same"))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization(axis=chanDim))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))

#   print("[INFO] 4...")
#   # first (and only) set of FC => RELU layers
#   model.add(Flatten())
#   model.add(Dense(1024))
#   model.add(Activation("relu"))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.5))

#   print("[INFO] 5...")
#   # softmax classifier
#   model.add(Dense(classes))
#   model.add(Activation("softmax"))

#   # return the constructed network architecture
#   return model


In [0]:
# def buildModel(width, height, depth, classes):
#   # initialize the model along with the input shape to be
#   # "channels last" and the channels dimension itself
#   model = Sequential()
#   inputShape = (height, width, depth)
#   chanDim = -1

#   # if we are using "channels first", update the input shape
#   # and channels dimension
#   if K.image_data_format() == "channels_first":
#     inputShape = (depth, height, width)
#     print("yes")
#     chanDim = 1

#   print("[INFO] 1...")
#   model.add(Dense(units=32, activation='relu', input_shape=inputShape))
#   model.add(Dense(classes))
#   model.add(Activation("softmax"))
  

#   # return the constructed network architecture
#   return model

In [0]:
# initialize the number of epochs, learning rate, batch size, and image dimensions
EPOCHS = 20
# we take 1e-3 because for adam optimizer this is the best learning rate
LEARNING_RATE = 1e-3
# NOTE: VERY IMP : Batch size must be less than number of images
BATCH_SIZE = 32
# image (height, width, depth)
IMAGE_DIMENSIONS = (96, 96, 3)

# data and labels lists
data = []
labels = []

In [0]:

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images("/content/gdrive/My Drive/E4Colaboratory/cnn/dataset")))
random.seed(42)
random.shuffle(imagePaths)
# print(imagePaths)

[INFO] loading images...


In [0]:
# loop over the input images
for imagePath in tqdm(imagePaths):
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMENSIONS[1], IMAGE_DIMENSIONS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

100%|██████████| 1433/1433 [02:00<00:00,  5.39it/s]


In [0]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

# construct the image generator for data augmentation
trainingDataAugmentation = ImageDataGenerator(rotation_range=30, 
                                              width_shift_range=0.1,
                                              height_shift_range=0.1,
                                              shear_range=0.2,
                                              zoom_range=0.2,
                                              horizontal_flip=True,
                                              fill_mode="nearest")


[INFO] data matrix: 309.53MB


In [0]:
# initialize the model
print("[INFO] compiling model...")
print("[INFO] number of classes: ", len(lb.classes_))
model = buildModel(width=IMAGE_DIMENSIONS[1], height=IMAGE_DIMENSIONS[0],
	depth=IMAGE_DIMENSIONS[2], classes=len(lb.classes_))
optimizer = Adam(lr=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
# we are using categorical_crossentropy because targets are one-hot encoded ie. [1,0,0],[0,1,0] 
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
	metrics=["accuracy"])
model.summary()


[INFO] compiling model...
[INFO] number of classes:  27
[INFO] 1...
[INFO] 2...
[INFO] 3...
[INFO] 4...
[INFO] 5...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
activation_8 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
__________________________

In [0]:
# train the network
print("[INFO] training network...")
print("[INFO] train_X  size...",trainX.shape)
print("[INFO] train_Y  size...",trainY.shape)

[INFO] training network...
[INFO] train_X  size... (1146, 96, 96, 3)
[INFO] train_Y  size... (1146, 27)


In [0]:
# IMAGEDATAGENERATORNOTAPPLIEDTOVALIDATIONDATA??????????!!!!!!!!!!?
H = model.fit_generator(
	trainingDataAugmentation.flow(trainX, trainY, batch_size=BATCH_SIZE),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BATCH_SIZE,
	epochs=EPOCHS, verbose=1)

Epoch 1/20
35/35 [==============================] - 6s 166ms/step - loss: 2.5327 - acc: 0.3804 - val_loss: 1.0102 - val_acc: 0.6794
Epoch 2/20
35/35 [==============================] - 3s 94ms/step - loss: 1.1174 - acc: 0.6675 - val_loss: 0.8139 - val_acc: 0.7700
Epoch 3/20
35/35 [==============================] - 4s 100ms/step - loss: 0.7540 - acc: 0.7699 - val_loss: 0.6896 - val_acc: 0.7979
Epoch 4/20
35/35 [==============================] - 3s 100ms/step - loss: 0.5060 - acc: 0.8318 - val_loss: 0.4156 - val_acc: 0.8641
Epoch 5/20
35/35 [==============================] - 4s 100ms/step - loss: 0.4755 - acc: 0.8595 - val_loss: 0.4604 - val_acc: 0.8676
Epoch 6/20
35/35 [==============================] - 3s 99ms/step - loss: 0.4101 - acc: 0.8755 - val_loss: 0.3920 - val_acc: 0.8850
Epoch 7/20
35/35 [==============================] - 3s 97ms/step - loss: 0.3647 - acc: 0.8802 - val_loss: 0.3440 - val_acc: 0.8920
Epoch 8/20
35/35 [==============================] - 3s 97ms/step - loss: 0.2936

In [0]:
# save the model to disk
print("[INFO] serializing network...")
model.save("/content/gdrive/My Drive/E4Colaboratory/cnn/products.model")


[INFO] serializing network...


In [0]:
print("[INFO] serializing label binarizer...")
f = open("/content/gdrive/My Drive/E4Colaboratory/cnn/lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing label binarizer...


In [0]:
history_dict = H.history
print("[INFO] key names...", history_dict.keys())

[INFO] key names... dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [0]:
# plot the training loss and accuracy

plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("/content/gdrive/My Drive/E4Colaboratory/cnn/TrainingLossAndAccuracyPlot_Epochs"+str(N)+"_Classes"+str())

In [0]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N),  H.history["acc"], 'bo', label='Training accuracy')
plt.plot(np.arange(0, N),  H.history["val_acc"], 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig("/content/gdrive/My Drive/E4Colaboratory/cnn/TrainingAndValidationAccuracyPlot_Epochs"+str(N)+"_Classes"+str())

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N),  H.history["loss"], 'bo', label='Training loss')
plt.plot(np.arange(0, N),  H.history["val_loss"], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig("/content/gdrive/My Drive/E4Colaboratory/cnn/TrainingAndValidationLossPlot_Epochs"+str(N)+"_Classes"+str())

In [0]:
plt.close('all')

In [0]:
# TESTING___________________________________________________________________________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [0]:
IMG_PATH = "/content/gdrive/My Drive/E4Colaboratory/cnn/examples/delmonte.jpg"
image = cv2.imread(IMG_PATH)
output = image.copy()


In [0]:
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [0]:
# load the trained convolutional neural network and the label
# binarizer
print("[INFO] loading network...")
model = load_model("/content/gdrive/My Drive/E4Colaboratory/cnn/products.model")
lb = pickle.loads(open("/content/gdrive/My Drive/E4Colaboratory/cnn/lb.pickle", "rb").read())



In [0]:
# classify the input image
print("[INFO] classifying image...")
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

[INFO] classifying image...


In [0]:
# cv2.imshow is disabled in Colab
# If you would like to re-enable this function, first run:
import os
os.environ["ENABLE_CV2_IMSHOW"] = 'true'

In [0]:
# we'll mark our prediction as "correct" of the input image filename
# contains the predicted label text (obviously this makes the
# assumption that you have named your testing image files this way)

filename = IMG_PATH[IMG_PATH.rfind(os.path.sep) + 1:]
correct = "correct" if filename.rfind(label) != -1 else "incorrect"

# build the label and draw the label on the image
label = "{}: {:.2f}% ({})".format(label, proba[idx] * 100, correct)
output = imutils.resize(output, width=400)
# cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# show the output image
print("[INFO] {}".format(label))
# cv2.imshow("Output", output)
# cv2.waitKey(0)

[INFO] del.monte tomato ketchup: 100.00% (incorrect)
